In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 910
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 195
    })
    test: Dataset({
        features: ['text'],
        num_rows: 195
    })
})


In [43]:
import json
data = json.loads(dataset['test'][1]['text'])

In [16]:
data['dialog'][6]['text']

'lot of people lose the job in this corona time'

In [47]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
rouge = evaluate.load('rouge')

In [52]:
messages = []

for i in range(1, len(data), 2):
    
    messages.append({"role": "user", "content":  data['dialog'][i]['text']})
    
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    
    outputs = model.generate(
            input_ids=input_ids,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
    )
    candidate = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    reference = data['dialog'][i+1]['text']
    candidates = candidate.split(" ")
    references = reference.split(" ")
    print("Reference: ", reference)
    print("Candidate: ", candidate)
    print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
    print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
    print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
    print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))
    print('Cumulative 1-gram: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))
    P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-base-uncased')

# Print the BERTScore
    print('Precision:', P)
    print('Recall:', R)
    print('F1 Score:', F1)
    results = rouge.compute(predictions=[candidate], references=[reference])
    print(results)
    messages.append({"role": "assistant", "content": candidate})
    

Reference:  ok tell me
Candidate:   Hello Tom, I'm here to help. What's been going on?
Individual 1-gram: 0.117647
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
Cumulative 1-gram: 0.000000
Precision: tensor([0.2919])
Recall: tensor([0.4304])
F1 Score: tensor([0.3479])
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
Reference:  ok, don't worry friend,
Candidate:   I can imagine how difficult and stressful it must be for you to be facing the possibility of losing your job during these uncertain times. It's completely understandable that you're feeling overwhelmed and worried about the future.
Individual 1-gram: 0.051724
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
Cumulative 1-gram: 0.000000
Precision: tensor([0.2510])
Recall: tensor([0.4093])
F1 Score: tensor([0.3111])
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
Reference:  lot of people lose the job in this corona time
Candi

In [20]:
import json

json_file_path = '../../ExTES.json'  # Change 'output.json' to your desired file name

# Write the list to the JSON file
with open(json_file_path, 'rb') as json_file:
    extes_data = json.load(json_file)

In [54]:
data = extes_data[0]['content']

In [22]:
data[0]['User']

'Hey, are you free? I need someone to talk to.'

In [39]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score


In [55]:
messages = []

for i in range(0, len(data), 2):
    
    messages.append({"role": "user", "content":  data[i]['User']})
    
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    
    outputs = model.generate(
            input_ids=input_ids,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
    )
    candidate = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    reference = data[i+1]['AI']
    
    references = reference.split(" ")
    candidates = candidate.split(" ")
    print("Reference: ", reference)
    print("Candidate: ", candidate)
    smoothie = SmoothingFunction().method4

# Calculate BLEU scores with smoothing
    print('Individual 1-gram: %f' % sentence_bleu(references, candidates, weights=(1, 0, 0, 0), smoothing_function=smoothie))
    print('Individual 2-gram: %f' % sentence_bleu(references, candidates, weights=(0, 1, 0, 0), smoothing_function=smoothie))
    print('Individual 3-gram: %f' % sentence_bleu(references, candidates, weights=(0, 0, 1, 0), smoothing_function=smoothie))
    print('Individual 4-gram: %f' % sentence_bleu(references, candidates, weights=(0, 0, 0, 1), smoothing_function=smoothie))
    print('Cumulative 1-gram: %f' % sentence_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie))
    P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-base-uncased')

# Print the BERTScore
    print('Precision:', P)
    print('Recall:', R)
    print('F1 Score:', F1)
    results = rouge.compute(predictions=[candidate], references=[reference])
    print(results)

    messages.append({"role": "assistant", "content": reference})
    

Reference:  Of course! I'm here to listen. What's been going on?
Candidate:   Of course! I'm here to listen. What's been going on?
Individual 1-gram: 0.000000
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
Cumulative 1-gram: 0.000000
Precision: tensor([1.])
Recall: tensor([1.])
F1 Score: tensor([1.])
{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
Reference:  That sounds really tough. It's completely normal to feel confused and insecure when a situation like this arises. Have you had any discussions about needing space in the past?
Candidate:  I can understand how challenging and painful that must be for you. It sounds like you're feeling confused and hurt.
Individual 1-gram: 0.050000
Individual 2-gram: 0.015767
Individual 3-gram: 0.008321
Individual 4-gram: 0.004405
Cumulative 1-gram: 0.013039
Precision: tensor([0.5615])
Recall: tensor([0.4799])
F1 Score: tensor([0.5175])
{'rouge1': 0.27450980392156865, 'rouge2': 0.04081632653061224